## Get DenseNet bottleneck features with pretrained model in PyTorch

In [ ]:
import os
from sklearn.datasets import load_files  
from keras.preprocessing import image 
import numpy as np
from keras.utils import np_utils

import torch
from torch.autograd import Variable
from torch import Tensor
import torch.nn as nn
from torchvision import transforms
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Convert image paths to torch variables
# i.e., 'channel' is the 2nd dimention, not the last

def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

def path_to_tonsor(img_path):
    img = image.load_img(img_path)
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return preprocess(img).unsqueeze(0)

def paths_to_tonsor(img_paths):
    list_of_tonsors = [path_to_tonsor(img_path) for img_path in img_paths]
    tonsors = torch.cat(list_of_tonsors, 0)
    variable = Variable(tonsors)
    return variable

In [ ]:
# Use Densenet from torchvision.models 
# load the model together with its pretrained weights
from torchvision.models import densenet161
densenet161 = densenet161(pretrained = True)
# remove last fully-connected layer
new_classifier = nn.Sequential(*list(densenet161.classifier.children())[:-1])
densenet161.classifier = new_classifier

### Pass forward the images to get the bottleneck feature maps

Due to the limit capacity of my laptop, passing too many images to the DenseNet network at once would stop the kernel. So I send each time a batch of 100 images, save the results in a temporary folder, then restart the kernel for the next batch. After doing this for 67 (9 and 9 respectively) times, for the train set (validation set and test set respectively), I got all the feature maps needed and concatenating them respectively gives the three *.npy* files in the directory of *bottleneck_features*, namely, *DogDenseNet_train*, *DogDenseNet_valid* and *DogDenseNet_test*

In [ ]:
test_files, test_targets = load_dataset('dogImages/test')
testBatch_tonsors = paths_to_tonsor(test_files[800:])
testBatch_DenseNet = densenet161.forward(testBatch_tonsors)
np.save('TestFeatureBatchs/Batch9.npy', testBatch_DenseNet.data.numpy())
os._exit(00)

In [ ]:
DogDenseNet_test = np.load('TestFeatureBatchs/Batch1.npy')
for i in range(2, 10):
    DogDenseNet_test = np.concatenate((DogDenseNet_test, np.load('TestFeatureBatchs/Batch'+str(i)+'.npy')))
np.save('bottleneck_features/DogDenseNet_test.npy', DogDenseNet_test)

## Now do the same *withOUT* PyTorch, since PyTorch cannot be used on a Window machine

I found the following implementations of DenseNet in Keras:

* The model in [repository](https://github.com/flyyufelix/DenseNet-Keras.git), stopped at the weight loading stage, due to errors saying that dimensions do not match. 
* The model in [repository](https://github.com/titu1994/DenseNet.git) can successfully return bottleneck feature maps after modifications:
    - was: ~~requie_flatten = include_top~~
    + is : include_top = include_top
    - was: ~~default_size = 32~~
    + is : default_size = 224

In [18]:
import densenet

image_dim = (224, 224, 3)
DenseNet_second = densenet.DenseNetImageNet161(input_shape=image_dim, include_top=False)

Weights for the model were loaded successfully


In [19]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
DenseNet_second.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])
DenseNet_output = DenseNet_second.predict(path_to_tensor('images/American_water_spaniel_00648.jpg'))
print(DenseNet_output.shape)

(1, 2208)
